# 4단계: 배포 파이프라인 추가하기
이전 네 단계에서 자동화된 데이터 처리 및 모델 구축 파이프라인을 구현했습니다. 파이프라인의 각 실행은 새로운 버전의 모델을 생성합니다. 이 노트북은 ML 워크플로우에서 자동화된 모델 배포 단계를 구현합니다.

![](img/sagemaker-mlops-project-deploy-diagram.jpg)

[SageMaker MLOps 프로젝트 템플릿](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates.html)을 사용하여 바로 사용할 수 있는 모델 배포 CI/CD 파이프라인을 프로비저닝할 수 있습니다.

이 템플릿은 SageMaker 모델 레지스트리의 모델을 실시간 추론을 위한 SageMaker 엔드포인트에 배포하는 과정을 자동화합니다. 이 템플릿은 모델 레지스트리의 변경 사항을 인식합니다. 새 모델 버전이 등록되고 승인되면 자동으로 배포를 시작합니다.

<div class="alert alert-info"> 이 노트북에서는 JupyterLab의 <code>Python 3</code> 커널을 사용하고 있는지 확인하세요.</div>

먼저, 이 노트북에 필요한 Python 종속성을 설치해야 합니다.

In [ ]:
%pip install jsonlines tqdm

In [ ]:
import boto3
import sagemaker 
from time import gmtime, strftime, sleep
import json
import os
from sagemaker.predictor import Predictor
import pandas as pd
from tqdm import trange
import numpy as np
sagemaker.__version__

In [ ]:
%store -r

In [ ]:
%store

In [ ]:
assert len(model_package_group_name) > 0
assert len(region) > 0
assert len(bucket_name) > 0
assert len(bucket_prefix) > 0

In [ ]:
sm = boto3.client("sagemaker")

In [ ]:
try:
    print(project_name)
    print(project_id)
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("You must complete the notebook 06-sagemaker-project")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++")

## Studio UI에서 프로젝트 살펴보기

<div class="alert alert-info">6단계 노트북을 실행하고 MLOps 프로젝트를 성공적으로 프로비저닝하고 프로젝트 파이프라인을 최소 한 번 실행했는지 확인하세요.</div>

다음 코드 셀에서 생성된 링크를 클릭하여 Studio UI에서 프로젝트와 모델 패키지를 확인하세요. 모델 패키지 그룹에 등록된 모델 버전이 최소 하나 이상 있어야 합니다.

In [ ]:
# check that the project exists
project_data = sm.describe_project(ProjectName=project_name)

In [ ]:
assert project_data['ProjectStatus'] == 'CreateCompleted', 'Project must be created at this point!'

In [ ]:
# check that at least one model version is registered in the model registry
model_packages = sm.list_model_packages(
    ModelPackageGroupName=f'{project_name}-{project_id}',
    ModelApprovalStatus='PendingManualApproval')

In [ ]:
assert len(model_packages['ModelPackageSummaryList']) > 0, 'You must have at least one model version in the status PendingManualApproval'

## 모델 배포를 위한 MLOps 프로젝트 작업하기
이 템플릿은 모델 배포 단계를 지정하는 구성 파일, 인프라로서의 엔드포인트를 정의하는 AWS CloudFormation 템플릿, 그리고 엔드포인트 테스트를 위한 시드 코드가 포함된 CodeCommit 리포지토리를 프로비저닝합니다.

이 템플릿은 다음과 같은 리소스를 제공합니다:

1. 스테이징 및 프로덕션 환경에 모델을 엔드포인트에 배포하는 템플릿 코드가 포함된 AWS CodeCommit 리포지토리
2. `source`, `build`, `deploy-to-staging`, `deploy-to-production` 단계가 있는 AWS CodePipeline 파이프라인. `source` 단계는 CodeCommit 리포지토리를 가리키고, `build` 단계는 해당 리포지토리에서 코드를 가져와 배포할 CloudFormation 스택을 생성합니다. `deploy-to-staging` 및 `deploy-to-production` 단계는 각 환경에 CloudFormation 스택을 배포합니다. 스테이징과 프로덕션 빌드 단계 사이에는 수동 승인 단계가 있어 MLOps 엔지니어가 프로덕션에 배포하기 전에 모델을 승인해야 합니다.
3. 모델 패키지 버전이 승인되거나 거부될 때 CodePipeline 파이프라인 실행을 시작하는 Amazon EventBridge 규칙.
4. 자리표시자 단위 테스트 이후에도 수동 승인 단계가 있습니다. 자리표시자 테스트를 대체하여 자체 테스트를 구현할 수 있습니다.

이 템플릿은 또한 CodePipeline 및 CodeBuild 아티팩트, SageMaker 파이프라인 실행에서 생성된 아티팩트를 포함한 아티팩트를 저장하기 위한 Amazon S3 버킷을 배포합니다.

다음 다이어그램은 아키텍처를 보여줍니다.

<img src="img/mlops-model-deploy.png" width="600"/>

프로젝트에 대한 구성 변경을 구현할 필요가 없습니다. 모델 배포 파이프라인은 즉시 작동합니다.
모델 배포 파이프라인을 시작하려면 모델 레지스트리에서 모델 버전을 승인해야 합니다.

### 모델 버전 승인하기
모델 버전을 승인하면 MLOps 프로젝트가 모델 배포 프로세스를 시작합니다.

첫 번째 단계에서 모델 배포 파이프라인은 모델 버전을 스테이징 SageMaker 실시간 추론 엔드포인트에 배포합니다.

Studio의 모델 레지스트리에서 또는 노트북에서 프로그래밍 방식으로 모델 버전을 승인할 수 있습니다. 프로그래밍 방식으로 해보겠습니다.

In [ ]:
try:
    print(model_package_group_name)
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Run the step 03 notebook to create a pipeline, run the pipeline, and register a model version in the model registry")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
# list all model packages and select the latest one
model_packages = []

for p in sm.get_paginator('list_model_packages').paginate(
        ModelPackageGroupName=model_package_group_name,
        SortBy="CreationTime",
        SortOrder="Descending",
    ):
    model_packages.extend(p["ModelPackageSummaryList"])

if len(model_packages) == 0:
    raise Exception(f"No model package is found for {model_package_group_name} model package group. Run a model creation pipeline first.")

print(f"There are {len(model_packages)} model versions in the {model_package_group_name} model package group")
print(f"Approve the most recent model package:")

latest_model_package_arn = model_packages[0]["ModelPackageArn"]
print(latest_model_package_arn)

다음 명령문은 모델 레지스트리에서 가장 최근 모델 패키지의 `ModelApprovalStatus`를 `Approved`로 설정합니다. 모델 패키지 상태 변경은 EventBridge 규칙을 시작하고, 이 규칙은 모델 배포가 포함된 CodePipeline CI/CD 파이프라인을 시작합니다.

In [ ]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=latest_model_package_arn,
    ModelApprovalStatus="Approved",
)

Studio UI의 모델 레지스트리에서 마지막 모델 버전의 **Status**가 `Approved`로 변경된 것을 확인할 수 있습니다:

![](img/model-package-group-version-approval.jpg)

### 배포 파이프라인 실행
위의 코드 셀에서 모델 버전이 승인되면 모델 배포 CI/CD 파이프라인은 다음 작업을 수행합니다:
1. SageMaker 엔드포인트 IaC가 포함된 CloudFormation 템플릿에 대한 스테이징 및 프로덕션 매개변수가 있는 CloudFormation 매개변수 구성 파일 생성
1. 현재 계정에 `<PROJECT-NAME>-staging` 이름의 SageMaker 실시간 추론 엔드포인트 생성
1. 스테이징 엔드포인트에서 테스트 스크립트 실행
1. [AWS CodePipeline 콘솔](https://console.aws.amazon.com/codesuite/codepipeline)에서 테스트 결과가 수동으로 승인될 때까지 대기
1. 현재 계정에 `<PROJECT-NAME>-prod` 이름의 SageMaker 엔드포인트 생성

파이프라인이 스테이징 엔드포인트 배포를 완료할 때까지 약 10-15분 정도 기다리세요. Studio UI의 **Deployments** > **Endpoints**에서 엔드포인트 상태를 확인할 수 있습니다:

![](img/sagemaker-mlops-deploy-endpoint-status.jpg)

엔드포인트 상태가 `Creating`에서 `InService`로 변경되면 스테이징 엔드포인트가 완전히 작동합니다. CodePipeline 파이프라인의 **DeployStaging** 단계를 수동으로 승인하여 프로덕션 단계로의 모델 배포 프로세스를 시작할 수 있습니다. 다음 섹션에서는 모델 배포를 승인하고 프로덕션 엔드포인트로의 배포 두 번째 단계를 시작합니다.

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">스테이징 엔드포인트 상태가 InService로 변경될 때까지 기다린 후 다음 코드 셀을 계속 진행하세요.
    </p>
</div>

# SageMaker 엔드포인트 테스트하기
스테이징에서 SageMaker 엔드포인트로의 성공적인 배포 후, 몇 가지 추론을 실행하여 엔드포인트를 검증해 보겠습니다.
모델을 엔드포인트에서 제공할 때 Sagemaker는 다양한 옵션을 제공합니다:

## SageMaker 엔드포인트
SageMaker는 배포의 기술적 범위와 깊이에 대한 선택권을 제공하며 추론 사용 사례에 대한 다양한 옵션을 제공합니다:

* **모델을 엔드포인트에 배포하기.** 모델을 배포할 때 다음 옵션을 고려하세요:
   + [실시간 추론](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html). 실시간 추론은 대화형, 낮은 지연 시간 요구사항이 있는 추론 워크로드에 이상적입니다.
   + [Amazon SageMaker Serverless Inference로 모델 배포](https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints.html). Serverless Inference를 사용하여 기본 인프라를 구성하거나 관리하지 않고 모델을 배포하세요. 이 옵션은 트래픽 급증 사이에 유휴 기간이 있고 콜드 스타트를 허용할 수 있는 워크로드에 이상적입니다.
   + [비동기 추론](https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference.html). 들어오는 요청을 대기열에 넣고 비동기적으로 처리합니다. 이 옵션은 큰 페이로드 크기(최대 1GB), 긴 처리 시간(최대 한 시간), 그리고 거의 실시간 지연 요구사항이 있는 요청에 이상적입니다.

* **비용 최적화**. 추론 비용을 최적화하려면 다음 옵션을 고려하세요:

   + [Amazon SageMaker 모델 자동 확장](https://docs.aws.amazon.com/sagemaker/latest/dg/endpoint-auto-scaling.html). 자동 확장을 사용하여 들어오는 트래픽 패턴에 따라 엔드포인트의 컴퓨팅 리소스를 동적으로 조정하세요. 이는 특정 시점에 사용 중인 리소스에 대해서만 비용을 지불하므로 비용을 최적화하는 데 도움이 됩니다.
 
다음 다이어그램은 SageMaker의 모든 배포 옵션에 대한 개요를 제공합니다:

![](img/sagemaker-deployment-modes.jpg)

## SageMaker 엔드포인트에서의 실시간 추론
추론 기능을 시연하기 위해 이 노트북에서는 실시간 추론과 배치 변환을 모두 살펴보겠습니다. 프로덕션 환경에 배포하기 전에 엔드포인트가 예상대로 작동하는지 확인하기 위해 테스트에 스테이징 엔드포인트를 사용할 것입니다.

In [ ]:
# List all deployed real-time endpoints
endpoints = sm.list_endpoints(StatusEquals="InService")["Endpoints"]

if not len(endpoints):
    print("There are no deployed active endpoints. You must have at least one endpoint. Run the previous cell in this notebook to deploy an endpoint")
else:
    print(f"Found {len(endpoints)} active inference endpoint(s):\n")
    
    for i, endpoint in enumerate(endpoints, 1):
        print(f"{i}. Endpoint Name: {endpoint['EndpointName']}")
        print(f"   Status: {endpoint['EndpointStatus']}")
        print(f"   Creation Time: {endpoint['CreationTime']}")
        print(f"   Last Modified: {endpoint['LastModifiedTime']}")
        print("---")
    
    # If you still need to select one endpoint for further operations, 
    # you can use the first one or add selection logic:
    endpoint_name = endpoints[0]['EndpointName']
    print(f"\nUsing endpoint '{endpoint_name}' for subsequent operations.")


> [!NOTE]
> 강사 주도 교육으로 이 워크숍을 진행하는 경우, `endpoint_name`이 이미 설정되어 있으므로 별도로 할 일이 없습니다. 그렇지 않은 경우, 테스트에 적합한 endpoint_name으로 업데이트하세요.

### 헬퍼 함수 정의하기
이 노트북 전체에서 사용할 코드 스니펫이 포함된 몇 가지 헬퍼 함수를 정의합니다.

In [ ]:
# Send data to the endpoint
def realtime_prediction(predictor, data):
    l = len(data)
    for i in trange(l):
        data_arr = [float(np_float) for np_float in data.iloc[i].values ]
        predictions = np.array(predictor.predict(data_arr), dtype=float).squeeze()
        print(predictions)

def download_from_s3(s3_client, local_file_path, bucket_name, s3_file_path):
    try:
        # Download the file
        s3_client.download_file(bucket_name, s3_file_path, local_file_path)
        print(f"File downloaded successfully to {local_file_path}")
        return True
    except ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            print(f"An error occurred: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

def upload_to_s3(s3_client, local_file_path, bucket_name, s3_file_path=None):
    # If S3 file path is not specified, use the basename of the local file
    if s3_file_path is None:
        s3_file_path = os.path.basename(local_file_path)

    try:
        # Upload the file
        s3_client.upload_file(local_file_path, bucket_name, s3_file_path)
        print(f"File {local_file_path} uploaded successfully to {bucket_name}/{s3_file_path}")
        return True
    except ClientError as e:
        print(f"ClientError: {e}")
        return False
    except FileNotFoundError:
        print(f"The file {local_file_path} was not found")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False
        
def write_params(s3_client, step_name, params, notebook_param_s3_bucket_prefix):
    local_file_path = f"{step_name}.json"
    with open(local_file_path, "w") as f:
        f.write(json.dumps(params))
    base_local_file_path = os.path.basename(local_file_path)
    bucket_name = notebook_param_s3_bucket_prefix.split("/")[2] # Format: s3://<bucket_name>/..
    s3_file_path = os.path.join("/".join(notebook_param_s3_bucket_prefix.split("/")[3:]), base_local_file_path)
    upload_to_s3(s3_client, local_file_path, bucket_name, s3_file_path)
    
def read_params(s3_client, notebook_param_s3_bucket_prefix, step_name):
    local_file_path = f"{step_name}.json"
    base_local_file_path = os.path.basename(local_file_path)
    bucket_name = notebook_param_s3_bucket_prefix.split("/")[2] # Format: s3://<bucket_name>/..
    s3_file_path = os.path.join("/".join(notebook_param_s3_bucket_prefix.split("/")[3:]),  base_local_file_path)
    downloaded = download_from_s3(s3_client, local_file_path, bucket_name, s3_file_path)
    with open(local_file_path, "r") as f:
        data = f.read()
        params = json.loads(data)
    return params

### 테스트 데이터를 기반으로 실시간 예측 생성하기
다음 셀에서는 이전 노트북에서 캡처한 테스트 데이터를 사용하여 실시간으로 몇 가지 추론을 실행하겠습니다.

In [ ]:
# Create a predictor class for the endpoint
predictor = Predictor(
    endpoint_name=endpoint_name, 
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer()
)

In [02-preprocess.ipynb](02-preprocess.ipynb) we divided the dataset into training, validation and test dataset. For this lab, we'll use the test dataset for running inferences against the deployed endpoint. 

In [ ]:
preprocess_step_name = "02-preprocess"
s3_client = boto3.client("s3", region_name=region)
notebook_param_s3_bucket_prefix=f"s3://{bucket_name}/{bucket_prefix}/params"
preprocess_step_params = read_params(s3_client, notebook_param_s3_bucket_prefix, preprocess_step_name)

In [ ]:
local_test_x_file_path = "test_x.csv"
local_test_y_file_path = "test_y.csv"
s3_test_x_data = preprocess_step_params["test_x_data"]
s3_test_y_data = preprocess_step_params["test_y_data"]

# Download the test_x.csv and test_y.csv file from S3
bucket_name = s3_test_x_data.split("/")[2]
s3_test_x_data_key = "/".join(s3_test_x_data.split("/")[3:])
s3_test_y_data_key = "/".join(s3_test_y_data.split("/")[3:])
download_from_s3(s3_client, local_test_x_file_path, bucket_name, s3_test_x_data_key)
download_from_s3(s3_client, local_test_y_file_path, bucket_name, s3_test_y_data_key)

In [ ]:
# Set the number of data vectors from the test dataset sent to the inference endpoint as batch
number_of_vectors = 10
test_x = pd.read_csv("test_x.csv", header=None).sample(number_of_vectors)

# Select only the first 10 features (columns 0-9) to match the trained model
test_x = test_x.iloc[:, :10]

In [ ]:
# Prints the output to see the response payload
print(f"Test data shape after feature selection: {test_x.shape}")
test_x.head()

Rename the column names for identifying the feature attributes

In [ ]:
# test_x will have 10 columns
test_x.columns = [f'_c{i}' for i in range(len(test_x.columns))]

Run prediction using the realtime endpoint deployed in the previous step

In [ ]:
realtime_prediction(predictor, test_x)

# Batch Transform
SageMaker offers [batch transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) to optimize inference workloads for the following  scenarios:

* Preprocess datasets to remove noise or bias that interferes with training or inference from your dataset.
* Get inferences from large datasets.
* Run inference when you don't need a persistent endpoint.
* Associate input records with inferences to help with the interpretation of results.

Functionally, batch transform uses the same mechanics as real-time hosting to generate predictions. It requires a web server that takes in HTTP POST requests a single observation, or mini-batch, at a time. However, unlike real-time hosted endpoints which have persistent hardware (instances stay running until you shut them down), batch transform clusters are torn down when the job completes.

To demonstrate the capability, we'll run a batch transform job on the same dataset that we used for realtime inference previously.

In [ ]:
# Download and preprocess the test data to use only first 10 features
import pandas as pd

# Download the original test data
local_test_x_batch_file = "test_x_batch_full.csv"
download_from_s3(s3_client, local_test_x_batch_file, bucket_name, s3_test_x_data_key)

# Load and select only first 10 features
test_x_full = pd.read_csv(local_test_x_batch_file, header=None)
test_x_batch = test_x_full.iloc[:, :10]  # Select only first 10 features

print(f"Original batch data shape: {test_x_full.shape}")
print(f"Modified batch data shape: {test_x_batch.shape}")

# Save the modified test data locally
local_test_x_batch_processed = "test_x_batch_processed.csv"
test_x_batch.to_csv(local_test_x_batch_processed, header=False, index=False)

# Upload the processed data to S3
s3_test_x_batch_processed_key = f"{bucket_prefix}/test/test_x_batch_processed.csv"
s3_test_x_batch_processed_path = f"s3://{bucket_name}/{s3_test_x_batch_processed_key}"

s3_client.upload_file(
    local_test_x_batch_processed, 
    bucket_name, 
    s3_test_x_batch_processed_key
)

print(f"Processed batch data uploaded to: {s3_test_x_batch_processed_path}")

In [ ]:
from sagemaker.transformer import Transformer
from sagemaker.inputs import TransformInput

먼저 배포된 SageMaker 엔드포인트에서 model_name을 가져오겠습니다.

In [ ]:
response = sm.describe_endpoint(
    EndpointName=endpoint_name
)
endpoint_config_name = response["EndpointConfigName"]
response = sm.describe_endpoint_config(EndpointConfigName=endpoint_config_name)
model_name = response['ProductionVariants'][0]['ModelName']

배치 변환 작업을 실행하기 위한 batch_transform 변수를 정의합니다.

In [ ]:
batch_transform_instance_type = "ml.m5.large"
batch_transform_output_path = f"s3://{bucket_name}/{bucket_prefix}/transform"
sagemaker_session = sagemaker.Session()

In [ ]:
# create the transform step
transformer = Transformer(
        model_name=model_name,
        instance_type=batch_transform_instance_type,
        instance_count=1,
        accept="text/csv",
        assemble_with="Line",
        output_path=batch_transform_output_path,
        sagemaker_session=sagemaker_session,
        base_transform_job_name=f"player-churn-model-batch-transform",
    )

SageMaker Python SDK를 사용하여 배치 변환 작업을 트리거합니다.

In [ ]:
# Use the processed data with only 10 features instead of original data
transformer.transform(    
        data=s3_test_x_batch_processed_path,  # Changed from s3_test_x_data
        content_type="text/csv",
        split_type="Line", 
        join_source="Input"
    )

S3 버킷에서 추론 결과를 다운로드합니다.

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./

페이로드를 살펴보겠습니다. 예측 값

> [!NOTE]
> 위의 결과는 입력 데이터와 CSV 형식의 예측이 포함된 응답 페이로드를 보여줍니다. 페이로드에는 입력과 예측 레이블이 모두 포함되어 있습니다. 기본 출력 구조 외에도 배치 변환이 출력을 구성하는 방식을 사용자 지정할 수 있습니다. 이러한 사용자 지정에 대해 자세히 알아보려면 이 [링크](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html#batch-transform-data-processing-examples)를 참조하세요.

In [ ]:
!head -10 test_x_batch_processed.csv.out

### 모델 버전을 프로덕션에 배포
스테이징 엔드포인트를 검증하고 결과에 만족했다고 가정하면, 이제 배포 파이프라인을 계속해서 프로덕션 배포를 진행하겠습니다.

CodePipeline 승인 링크를 생성해 보겠습니다.

옵션 1 `boto3`를 사용하여 MLOps 프로젝트를 생성한 경우, `project_name` 및 `project_id`가 자동으로 설정됩니다. 다음 코드 셀을 실행하여 값을 출력할 수 있습니다. UI 지침에 따라 프로젝트를 생성한 경우 `project_name`을 수동으로 설정해야 합니다.

In [ ]:
try:
    print(project_name)
    print(project_id)
except NameError:
    print("++++++++++++++++++++++++++++++++++++++")
    print("You must set the project_name manually")
    print("++++++++++++++++++++++++++++++++++++++")

In [ ]:
# Set to the model deployment project name if you didn't use boto3-based deployment
# project_name = "<PROJECT_NAME>"

# Get project id
project_id = sm.describe_project(ProjectName=project_name)['ProjectId']

# Construct the CodePipline pipeline name
code_pipeline_name = f"sagemaker-{project_name}-{project_id}-modeldeploy"

In [ ]:
 # approve the latest model version
model_package_update_response = sm.update_model_package(
    ModelPackageArn=latest_model_package_arn,
    ModelApprovalStatus="Approved",
)

In [ ]:
from IPython.display import HTML
# Show the approval link
display(
    HTML(
        '<b>Please approve the manual step in <a target="top" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/{}/view?region={}">AWS CodePipeline</a></b>'.format(
            code_pipeline_name, region)
    )
)

위의 ^^^ 링크 ^^^를 클릭하여 파이프라인 실행 워크플로우가 있는 CodePipeline 콘솔을 엽니다.

**DeployStaging 단계**에서 **ApproveDeployment** 단계의 **Review**를 선택합니다. `TestStaging` 단계가 `Succeeded` 상태로 완료될 때까지 기다려야 할 수 있습니다.

![](img/deploy-staging-review.png)

**Review** 대화 상자에서 **Approve**를 선택하고 **Submit**을 클릭합니다:

![](img/approve-deployment.png)

**DeployStaging** 단계를 승인하면 배포 파이프라인이 계속 진행되어 모델을 프로덕션 엔드포인트에 배포합니다. 엔드포인트를 보려면 Studio UI에서 **Deployments** > **Endpoints**를 선택하세요.

CI/CD 배포 파이프라인이 계속 진행됨에 따라 이전에 배포된 스테이징 엔드포인트가 `InService` 상태인 동안 프로덕션 엔드포인트가 `Creating` 상태인 것을 볼 수 있습니다:

![](img/endpoint-prod-creating.png)

`10-15`분 후 배포가 완료되고 두 엔드포인트 모두 `InService` 상태가 됩니다.

Studio로 이동하여 **Deployments** > **Projects**를 선택합니다. Project 창에서 `model-deploy-<TIMESTAMP>` 프로젝트를 선택합니다. 프로젝트 세부 정보 창에서 **Endpoints**를 선택합니다. 프로젝트와 엔드포인트가 메타데이터를 통해 연결되어 있기 때문에 `staging`과 `prod` 두 엔드포인트 모두 배포 프로젝트에 표시됩니다:

![](img/project-endpoints.png)

## 요약
이 노트북에서는 다음 기능을 갖춘 자동화된 CI/CD 배포 파이프라인을 구현했습니다:
- SageMaker 실시간 추론 엔드포인트 배포를 위한 CloudFormation IaC 템플릿 사용
- 모델 레지스트리의 모델 승인이 모델 배포 파이프라인을 시작
- 모델 배포 파이프라인에는 스테이징 엔드포인트에 대한 자동화된 테스트와 프로덕션 배포를 위한 수동 승인이 포함된 두 단계(스테이징 및 프로덕션)가 포함됨

## 정리
<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">
    6단계 노트북(데이터 및 모델 품질 모니터링)을 실행할 예정이라면 엔드포인트 중 하나 이상을 유지해야 합니다. 여기서 워크숍을 마치고 6단계 노트북을 실행하지 않을 경우, <b>정리 노트북(99-clean-up.ipynb)</b>으로 이동하여 정리 지침을 따라 AWS 계정에서 요금이 발생하지 않도록 하세요.
    <br>
    <br>
    AWS에서 제공한 AWS 계정을 사용하는 경우에는 정리를 실행할 필요가 없습니다.
    </p>
</div>

## 실제 프로젝트를 위한 추가 개발 아이디어
- AWS KMS 키를 사용한 엔드-투-엔드 데이터 암호화 추가
- 특정 프로젝트 요구 사항을 충족하기 위한 모델 배포용 [사용자 지정 SageMaker 프로젝트 템플릿](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-custom.html) 생성
- ML 워크플로우에 [다중 계정 모델 배포](https://aws.amazon.com/blogs/machine-learning/multi-account-model-deployment-with-amazon-sagemaker-pipelines/) 추가
- CodePipeline 파이프라인의 플레이스홀더에 자동화된 모델 테스트 추가
- [Amazon SageMaker Inference Recommender](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-recommender.html)를 사용하여 추론 엔드포인트에 대한 자동화된 부하 테스트를 실행하고 최적의 인스턴스 유형 및 구성 선택

## 추가 리소스
- [실시간 추론 엔드포인트에 머신 러닝 모델 배포](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-deploy-model-to-real-time-inference-endpoint/)
- [SageMaker MLOps 프로젝트 안내](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html)
- [SageMaker Immersion Day의 Amazon SageMaker Pipelines 랩](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab6)
- [Amazon SageMaker 보안 MLOps](https://github.com/aws-samples/amazon-sagemaker-secure-mlops)
- [Amazon SageMaker ML 모델을 위한 테스트 접근 방식](https://aws.amazon.com/blogs/machine-learning/testing-approaches-for-amazon-sagemaker-ml-models/)
- [Amazon SageMaker의 모델 호스팅 패턴 블로그 시리즈](https://aws.amazon.com/blogs/machine-learning/model-hosting-patterns-in-amazon-sagemaker-part-1-common-design-patterns-for-building-ml-applications-on-amazon-sagemaker/)
- [Amazon SageMaker 배포 가드레일을 활용한 고급 배포 전략](https://aws.amazon.com/blogs/machine-learning/take-advantage-of-advanced-deployment-strategies-using-amazon-sagemaker-deployment-guardrails/)
- [Amazon SageMaker를 사용한 실시간 추론 모델 서빙 엔드포인트를 위한 MLOps 배포 모범 사례](https://aws.amazon.com/blogs/machine-learning/mlops-deployment-best-practices-for-real-time-inference-model-serving-endpoints-with-amazon-sagemaker/)

# 커널 종료

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>